In [1]:
import pandas as pd, networkx as nx

In [2]:
## Reference network preparation 

HIPPIE_df=pd.read_csv("../Source/Interactomes/HIPPIE_v2_2.tab", keep_default_na=False, sep="\t")
print("The number of interaction in HIPPIE",len(HIPPIE_df.index))
HIPPIE_nx=nx.from_pandas_edgelist(HIPPIE_df,"Gene_1","Gene_2",edge_attr="Score")
self_interactions=list(nx.selfloop_edges(HIPPIE_nx))

# during permutation, we have to get rif off self interactions and nan-nan interaction, coming from reference network as a default.
for u, v in self_interactions:
    if isinstance(u, float):
        print(u)
    else:
        HIPPIE_nx.remove_edge(u,v)
    
#HIPPIE_nx.remove_edges_from(self_interactions)
HIPPIE_df=nx.to_pandas_edgelist(HIPPIE_nx)
print("The number of edges in HIPPIE network",HIPPIE_nx.number_of_edges())
print("The number of nodes in HIPPIE network",HIPPIE_nx.number_of_nodes())


edges=tuple(zip(HIPPIE_df.source,HIPPIE_df.target))
for edge in edges:
    if edge not in HIPPIE_nx.edges:
        print("check the edge",edge)

The number of interaction in HIPPIE 363336
The number of edges in HIPPIE network 345770
The number of nodes in HIPPIE network 15861


In [3]:
## initial nodes and weights 

initial_nodes_df=pd.read_csv(f'../Source/Netpath/Sampling_0_5/AndrogenReceptor_05A/AndrogenReceptor_05A_var_0.nodes',sep="\t")

initial_nodes=initial_nodes_df.name.to_list()
initial_weights=[1 for _ in initial_nodes]

In [4]:
from Paragon import GraphletFrequency 

## network permutation

In [5]:

GRF=GraphletFrequency(HIPPIE_nx,initial_nodes)

## network permutation once
Permutated_nx=GRF.permutate_network()
Permutated_df=nx.to_pandas_edgelist(Permutated_nx)
Permutated_df
GRF.save_permuted_network(f'../Source/Interactomes/Permuted/HIPPIE_permuted/HIPPIE_perm_4')

'../Source/Interactomes/Permuted/HIPPIE_permuted/HIPPIE_perm_4'

In [6]:
## Permutation of reference network multiple times (n permuted network)

GRF.save_permuted_networks(f_name=f'../Source/Interactomes/Permuted/HIPPIE_permuted/HIPPIE_prm',
                          size=100)

permuted network saved in	 ../Source/Interactomes/Permuted/HIPPIE_permuted/HIPPIE_prm_1
permuted network saved in	 ../Source/Interactomes/Permuted/HIPPIE_permuted/HIPPIE_prm_2


1

# Graphlet Frequencies & Graphlet Gudided Network Construction


we keep the statistical information of graphlets such as their interactions, frequencies etc


In [11]:
### Graphlet Frequencies

## frequencies from reference network

GRF=GraphletFrequency(HIPPIE_nx,initial_nodes)
Own_freq=pd.DataFrame([GRF.get_Graphlet_Frequency()])## GRF.get_Graphlet_Frequency() gives the frequencies in dict

### frequencies from permutated network 

Random_frequencies=GRF.get_frequencies_from_pool("../Source/Interactomes/Permuted/HIPPIE_permuted/")

#Random_frequencies


# Frequency calculations from permuted network might take long time depending on the network size. You can load the already calculated frequecies from pickle file 
#GRF.get_frequencies_from_pickle("../AndrogenReceptor_Pool_Frequencies.pickle")



In [12]:
GRF.get_Graphlet_Frequency()

{'Graphlets0': 0.0005076737368511812,
 'Graphlets1': 0.03773616140713916,
 'Graphlets2': 0.017971374374892223,
 'Graphlets3': 0.5223535092257285,
 'Graphlets4': 0.1784086911536472,
 'Graphlets5': 0.08090774271426109,
 'Graphlets6': 0.11248008277289188,
 'Graphlets7': 0.03954750819106743,
 'Graphlets8': 0.010087256423521297}

In [13]:
GRF.get_frequencies_from_pickle("../AndrogenReceptor_Pool_Frequencies.pickle")

### Motif Selection

We compare the frequencies of graphlets in reference network and its permutation

In [23]:
GRF.get_Z_score()

{'Graphlets0': (13.981109361276678, 1.0164711621330179e-44),
 'Graphlets1': (-6.062503804298664, 6.700931865275906e-10),
 'Graphlets2': (6.495700627660494, 4.13239318071793e-11),
 'Graphlets3': (-5.605081828684537, 1.0407829941031376e-08),
 'Graphlets4': (-1.0129980991920728, 0.15553053442595238),
 'Graphlets5': (5.564931147547997, 1.311279695793731e-08),
 'Graphlets6': (8.966223680736274, 1.5342723073451396e-19),
 'Graphlets7': (12.928286830521651, 1.558433414973421e-38),
 'Graphlets8': (11.280782284339463, 8.164371283260952e-30)}

#### Frequent graphlets

In [24]:
Selected_Graphlets=GRF.select_significant_Graphlets()

In [25]:
Selected_Graphlets.keys() # selected significant graphlets

dict_keys(['Graphlets0', 'Graphlets2', 'Graphlets5', 'Graphlets6', 'Graphlets7', 'Graphlets8'])

#### One by one, wee can list graphlets where  keys are composed of the node list and value is of the edge list.

In [17]:
Selected_Graphlets['Graphlets0']

{('APPL1', 'AR'): [('APPL1', 'AR')],
 ('APPL1', 'PIK3R2'): [('APPL1', 'PIK3R2')],
 ('APPL1', 'HSPB1'): [('APPL1', 'HSPB1')],
 ('APPL1', 'AKT1'): [('APPL1', 'AKT1')],
 ('APPL1', 'HDAC1'): [('APPL1', 'HDAC1')],
 ('GAPDH', 'AR'): [('GAPDH', 'AR')],
 ('GAPDH', 'MDM2'): [('GAPDH', 'MDM2')],
 ('GAPDH', 'PXN'): [('GAPDH', 'PXN')],
 ('GAPDH', 'JUN'): [('GAPDH', 'JUN')],
 ('GAPDH', 'AKT1'): [('GAPDH', 'AKT1')],
 ('GAPDH', 'RNF4'): [('GAPDH', 'RNF4')],
 ('GAPDH', 'SIRT1'): [('GAPDH', 'SIRT1')],
 ('GAPDH', 'BRCA1'): [('GAPDH', 'BRCA1')],
 ('GAPDH', 'CDK5'): [('GAPDH', 'CDK5')],
 ('GAPDH', 'ATF2'): [('GAPDH', 'ATF2')],
 ('SP1', 'AR'): [('SP1', 'AR')],
 ('SMAD3', 'SP1'): [('SP1', 'SMAD3')],
 ('MDM2', 'SP1'): [('SP1', 'MDM2')],
 ('SP1', 'JUN'): [('SP1', 'JUN')],
 ('SRC', 'SP1'): [('SP1', 'SRC')],
 ('SP1', 'RNF4'): [('SP1', 'RNF4')],
 ('SIRT1', 'SP1'): [('SP1', 'SIRT1')],
 ('DDX5', 'SP1'): [('SP1', 'DDX5')],
 ('AHR', 'SP1'): [('SP1', 'AHR')],
 ('NCOR1', 'SP1'): [('SP1', 'NCOR1')],
 ('SP1', 'BRCA1'): 

#### Greaphlet-Guided Network is composed of freqeuntly seen graphlets, motifs. Here, we can list edges in GGN. 

In [27]:
GRF.get_selected_edge_list()

[('APPL1', 'AR'),
 ('APPL1', 'PIK3R2'),
 ('APPL1', 'HSPB1'),
 ('APPL1', 'AKT1'),
 ('APPL1', 'HDAC1'),
 ('APPL1', 'TP53'),
 ('APPL1', 'EGFR'),
 ('APPL1', 'HDAC3'),
 ('APPL1', 'HDAC2'),
 ('APPL1', 'CTNNB1'),
 ('APPL1', 'PIK3R1'),
 ('APPL1', 'DCC'),
 ('APPL1', 'SOCS6'),
 ('APPL1', 'SH2D2A'),
 ('APPL1', 'VCP'),
 ('APPL1', 'NTRK1'),
 ('APPL1', 'GNAI1'),
 ('APPL1', 'CBLB'),
 ('APPL1', 'LAMP1'),
 ('APPL1', 'AKT2'),
 ('APPL1', 'IGF1R'),
 ('APPL1', 'CBL'),
 ('APPL1', 'OCRL'),
 ('APPL1', 'TGFBR1'),
 ('APPL1', 'PIK3CA'),
 ('APPL1', 'ACTN2'),
 ('APPL1', 'PRKCZ'),
 ('APPL1', 'UBE2N'),
 ('APPL1', 'TUBB3'),
 ('APPL1', 'RAB5A'),
 ('APPL1', 'EEA1'),
 ('APPL1', 'RHEBL1'),
 ('APPL1', 'TRAF6'),
 ('APPL1', 'RUVBL2'),
 ('APPL1', 'MBD3'),
 ('APPL1', 'DACT1'),
 ('APPL1', 'GATAD2B'),
 ('APPL1', 'RBBP7'),
 ('APPL1', 'HDAC5'),
 ('APPL1', 'RBBP4'),
 ('APPL1', 'MTA2'),
 ('APPL1', 'RTCB'),
 ('APPL1', 'LARP1'),
 ('APPL1', 'MAP3K1'),
 ('APPL1', 'MYO6'),
 ('APPL1', 'UBE2V1'),
 ('APPL1', 'TRAF2'),
 ('APPL1', 'UBA1'),
 

#### we can write GGN into sif file

In [28]:
GRF.write_created_network("../AndrogenReceptor_GGN")

True

In [29]:
#### The calculation of graphlet frequencies may take long time. Thus, we can save frequencies or we can calculate in an exemplified pool.  In the downstream part, we can manually decide required graphlets.


In [30]:
GRF.save_frequencies_into_pickle("../AndrogenReceptor_Pool_Frequencies")

In [31]:
GRF.get_frequencies_from_pickle("../AndrogenReceptor_Pool_Frequencies.pickle")

In [32]:
GRF.get_Graphlet_Frequency()

{'Graphlets0': 0.0005076737368511812,
 'Graphlets1': 0.03773616140713916,
 'Graphlets2': 0.017971374374892223,
 'Graphlets3': 0.5223535092257285,
 'Graphlets4': 0.1784086911536472,
 'Graphlets5': 0.08090774271426109,
 'Graphlets6': 0.11248008277289188,
 'Graphlets7': 0.03954750819106743,
 'Graphlets8': 0.010087256423521297}

In [33]:
GRF.get_Graphlet_Frequency()

{'Graphlets0': 0.0005076737368511812,
 'Graphlets1': 0.03773616140713916,
 'Graphlets2': 0.017971374374892223,
 'Graphlets3': 0.5223535092257285,
 'Graphlets4': 0.1784086911536472,
 'Graphlets5': 0.08090774271426109,
 'Graphlets6': 0.11248008277289188,
 'Graphlets7': 0.03954750819106743,
 'Graphlets8': 0.010087256423521297}

In [34]:
Graphlet_list=list(Selected_Graphlets.keys())
Graphlet_list

['Graphlets0',
 'Graphlets2',
 'Graphlets5',
 'Graphlets6',
 'Graphlets7',
 'Graphlets8']

#### GGN construction with known graphlet-motifs 





In [36]:
from Paragon import GraphletGuidance

In [37]:
GRF_lite=GraphletGuidance(HIPPIE_nx)

In [38]:

GGN_nx=GRF_lite.construct_GGN(initial_nodes,Graphlets=Graphlet_list,extention=False) # extension: True --> add existing interaction of hidden nodes in reference network 
## GGN_nx, Graph item in networkx


82 of 82 input nodes have been found in the given network



In [39]:
GGN_nx.number_of_nodes()

3246

In [40]:
GGN_df=nx.to_pandas_edgelist(GGN_nx)
GGN_df

,source,target
0,APPL1,AR
1,APPL1,TP53
2,APPL1,EGFR
3,APPL1,HDAC3
4,APPL1,HDAC2
...,...,...
13552,SF1,CIAO1
13553,H3C1; H3C2; H3C3; H3C4; H3C6; H3C7; H3C8; H3C1...,KDM4C
13554,H3-3A; H3-3B,KDM4C
13555,H3C15; H3C14; H3C13,KDM4C


In [41]:
len(initial_nodes)

82

### PageRank Flux Calculations

Without guidance network, we can directly infer subnetwork from reference network through PageRankFlux algorithm.


In [43]:

## Weights of initial nodes
initial_weights=[1 for _ in initial_nodes]

In [44]:
from Paragon import NetworkInference 

In [45]:
pgrf=NetworkInference(network=HIPPIE_nx,edge_attribute="Score")
pgrf.load_initial_nodes(list(initial_nodes))


##### max_edge_count =>  limit the number of edge, the maximum number of edges with the higest flux scores




##### alpha => Damping parameter for PageRank, the propbability of walking into neighbors 


##### threshold => the scaling factor, the threshold percentage of summed flux scores,
             the algırithm select the edges from high to low flux scores by summing up the scores till completing intrested percentage  



##### intermediate_only => True or False: 
                 True --> algorithm only select interactions in graphlet guided network between initial nodes.  
                 False --> algorithm select interactions in graphlet guided network unconsidering initial nodes.


In [46]:



returned_nx=pgrf.reconstruct_subnetwork( max_edge_count=2000,alpha=0.8,threshold=0.8,intermediate_only=True)




theshold of 0.087000 limits predictions to 2000 edges


In [47]:
returned_nx.number_of_edges()

1636

In [48]:
# the number of interactions that are not located in initial node to initial node paths
2000-returned_nx.number_of_edges() #

364

In [49]:
returned_df=nx.to_pandas_edgelist(returned_nx)
returned_df.to_csv('../AndrogenReceptor_recontructed_pathway_without_GGN.sif')

In [50]:
## Network inference with GraphletGudidedNetwork

In [51]:
pgrf=NetworkInference(network=HIPPIE_nx,guide_network=GGN_nx,edge_attribute="Score")
pgrf.load_initial_nodes(list(initial_nodes))


In [52]:

returned_nx=pgrf.reconstruct_subnetwork( max_edge_count=2000,alpha=0.8,threshold=0.8,intermediate_only=True)


theshold of 0.654000 limits predictions to 2000 edges


In [53]:
returned_nx.number_of_edges()

1799

In [54]:
# the number of interactions that are not located in initial node to initial node paths
2000-returned_nx.number_of_edges() #

201

In [55]:
returned_df=nx.to_pandas_edgelist(returned_nx)
returned_df.to_csv('../AndrogenReceptor_reconstructed_pathway_with_GGN.sif')

In [56]:
returned_df=pd.read_csv('../AndrogenReceptor_reconstructed_pathway_with_GGN.sif')
returned_nx=nx.from_pandas_edgelist(returned_df)
returned_nx.number_of_edges()

1799

# Community detection

In [5]:
from Paragon import CommunityAnalysis 

In [6]:
returned_df=pd.read_csv('../AndrogenReceptor_reconstructed_pathway_with_GGN.sif')
returned_nx=nx.from_pandas_edgelist(returned_df)
returned_nx.number_of_edges()

1799

In [7]:
CA=CommunityAnalysis(returned_nx)

In [8]:
module_df, node_df=CA.get_communities_in_DataFrames()
node_df

,Genes,Community
0,RUNX2,Module_1
1,KLF13,Module_1
2,SMAD2,Module_1
3,KAT2B,Module_1
4,MSX2,Module_1
...,...,...
356,PPP1CA,Module_37
357,RNF123,Module_37
358,FBXO6,Module_37
359,PTK2,Module_37


In [9]:
module_df

,Community_name,Community
0,Module_1,RUNX2;KLF13;SMAD2;KAT2B;MSX2;RB1;NFE4
1,Module_2,NCL;MDM2;FOXO1;SATB1;S100A2;FOXO4;ZNF550;MANF;...
2,Module_3,PNRC2;ATF2;POU2F2;GAPDH;CTNNB1;CEBPB;AR;SMARCA...
3,Module_4,FHL2;RELA;AHR;HSP90AA1;PSMC2;CDK9;PTEN;EGLN3;H...
4,Module_5,XPO1;SEZ6L2;DLX5;FOXA1;ONECUT1
5,Module_6,MYOCD;AKT1;SMAD3;APP;STUB1;NR3C1;HOXC11
6,Module_7,PML;SUMO3;SIRT6;SENP1;PARP1;ARRB2;GCM1
7,Module_8,ZMIZ2;SLC39A7;SSX2; SSX2B;ARSJ;PIN1;PIAS1;MCPH1
8,Module_9,TBP;PIAS3;TAB2;OPN1LW;PRPF40A
9,Module_10,MECP2;RNF4;MAF;ZNF707;RBAK;ZNF420


In [10]:
#### Hypergeometric test for GOA

In [11]:
GOA_bio_proc_df=pd.read_csv(f'../Source/Annotations/GOA_proteins_isoforms_prepared.tab',sep="\t")
GOA_bio_proc_df

,GO ID,DB Object Symbol
0,GO:0002250,IGKV3-7
1,GO:0002250,IGKV1D-42
2,GO:0002250,IGLV4-69
3,GO:0002250,IGLV8-61
4,GO:0002250,IGLV4-60
...,...,...
210592,GO:0006958,C1QA
210593,GO:0001682,RPP40
210594,GO:0061640,DCTN3
210595,GO:0045892,NELFCD


In [12]:
### hypergeometric test for a community

In [14]:
CA.hypergeometric_test_for_community("Module_36",
                                     reference_network=HIPPIE_nx,
                                           prior_knowledge_df=GOA_bio_proc_df,
                                           prior_knowledge_on="GO ID", 
                                           name_on="DB Object Symbol")

,GO ID,p-value,Erichment_Score,Genes in Module,Intersecting Genes,The number of intersecting genes,Process_Gene,The number of components of prior_knowledge,Community_name
0,GO:0000122,0.000080,2.598862,"[MEPCE, CHD4, H3C1, H3C2, H3C3, H3C4, H3C6...","[HEXIM1, CHD3, SNW1, CUL3, LARP7, MEPCE, RNF2,...",8,"[LINC-PINT, E2F8, FEZF1, CNOT1, NUPR2, HELT, N...",968,Module_36
1,GO:0006338,0.000009,4.291570,"[MEPCE, CHD4, H3C1, H3C2, H3C3, H3C4, H3C6...","[TOP1, CHD3, CHD4, SMARCE1, RNF2]",5,"[ARID1A, CHD1, BTAF1, KDM6B, KDM6A, ZNHIT1, SM...",187,Module_36
2,GO:0006357,0.509719,-1.002004,"[MEPCE, CHD4, H3C1, H3C2, H3C3, H3C4, H3C6...","[SNW1, SMARCE1, CDC5L]",3,"[MT-RNR1, EPOP, UNCX, ZNF98, ZNF716, ZNF724, Z...",1697,Module_36
3,GO:0045892,0.001960,2.448193,"[MEPCE, CHD4, H3C1, H3C2, H3C3, H3C4, H3C6...","[HEXIM1, CHD3, SNW1, CHD4, SMARCE1]",5,"[PRAMEF33, A0A0G2JP20, TLE7, PRAMEF22, PRAMEF2...",587,Module_36
4,GO:0045893,0.105222,1.030392,"[MEPCE, CHD4, H3C1, H3C2, H3C3, H3C4, H3C6...","[CHD3, CHD4, SMARCE1]",3,"[TAF11L5, TAF11L4, TAF11L13, TAF11L14, TAF11L1...",729,Module_36
5,GO:0045944,0.292603,-0.056268,"[MEPCE, CHD4, H3C1, H3C2, H3C3, H3C4, H3C6...","[SNW1, COPS5, CDC5L]",3,"[E2F8, ZBTB7C, NKX2-6, ZNF840P, HELT, MSGN1, N...",1200,Module_36


In [12]:
returned_all=CA.hypergeometric_test_for_all_communities(reference_network=HIPPIE_nx,
                                           prior_knowledge_df=GOA_bio_proc_df,
                                           prior_knowledge_on="GO ID", 
                                           name_on="DB Object Symbol")
returned_all 

,Community_name,GO ID,p-value,Erichment_Score,Genes in Module,Intersecting Genes,The number of intersecting genes,Process_Gene,The number of components of prior_knowledge
0,Module_1,GO:0006351,2.087920e-04,4.972020,"[RUNX2, KLF13, SMAD2, KAT2B, MSX2, RB1, NFE4]","[SMAD2, NFE4, KAT2B]",3,"[BCLAF3, LINC00473, AIP, TCERG1, POLR3A, BTAF1...",294
1,Module_1,GO:0006355,1.916913e-05,4.594412,"[RUNX2, KLF13, SMAD2, KAT2B, MSX2, RB1, NFE4]","[RB1, NFE4, KAT2B, RUNX2, SMAD2]",5,"[ZNF722, A0A2R8YD15, FOXO3B, NFILZ, A0A7P0TAN4...",1006
2,Module_1,GO:0006357,1.587277e-07,5.873635,"[RUNX2, KLF13, SMAD2, KAT2B, MSX2, RB1, NFE4]","[NFE4, KAT2B, RUNX2, KLF13, SMAD2, MSX2, RB1]",7,"[MT-RNR1, EPOP, UNCX, ZNF98, ZNF716, ZNF724, Z...",1697
3,Module_1,GO:0008285,1.262080e-05,5.276092,"[RUNX2, KLF13, SMAD2, KAT2B, MSX2, RB1, NFE4]","[MSX2, SMAD2, KAT2B, KLF13]",4,"[FEZF1, ZBTB7C, NUPR2, SULT2B1, MEN1, CACNB4, ...",396
4,Module_1,GO:0045892,5.945193e-05,4.675598,"[RUNX2, KLF13, SMAD2, KAT2B, MSX2, RB1, NFE4]","[RB1, MSX2, RUNX2, SMAD2]",4,"[PRAMEF33, A0A0G2JP20, TLE7, PRAMEF22, PRAMEF2...",587
...,...,...,...,...,...,...,...,...,...
236,Module_37,GO:0007165,1.278441e-02,2.115723,"[GSN, TGFB1I1, PXN, CRKL, PIK3R1, CBLB, NEDD8,...","[PXN, CRKL, CBLB, PIK3R1]",4,"[CCL4L2, A0A2R8Y747, A0A499FJF3, ARHGAP10, PSD...",1293
237,Module_37,GO:0016477,8.681411e-04,4.120524,"[GSN, TGFB1I1, PXN, CRKL, PIK3R1, CBLB, NEDD8,...","[PXN, PTK2, CRKL]",3,"[PRSS37, FOXE1, PODXL, ITGB1BP1, LAMA5, ARPC5,...",261
238,Module_37,GO:0016567,4.650220e-04,3.635703,"[GSN, TGFB1I1, PXN, CRKL, PIK3R1, CBLB, NEDD8,...","[CBLB, RNF123, FBXO6, NEDD8]",4,"[TMEM129, UBA6, ASB14, RFPL4A, MARCHF11, ZSWIM...",522
239,Module_37,GO:0034446,9.973064e-06,6.391448,"[GSN, TGFB1I1, PXN, CRKL, PIK3R1, CBLB, NEDD8,...","[ITGA4, PIK3R1, PXN]",3,"[NRP1, LAMA5, SRGAP2, FZD7, ABL1, FN1, ITGB3, ...",58


## significant modules in network

In [13]:
returned_all.Community_name.unique()

array(['Module_1', 'Module_2', 'Module_3', 'Module_4', 'Module_5',
       'Module_6', 'Module_7', 'Module_10', 'Module_11', 'Module_13',
       'Module_14', 'Module_15', 'Module_17', 'Module_18', 'Module_19',
       'Module_20', 'Module_21', 'Module_22', 'Module_23', 'Module_24',
       'Module_25', 'Module_26', 'Module_27', 'Module_28', 'Module_29',
       'Module_30', 'Module_32', 'Module_34', 'Module_35', 'Module_36',
       'Module_37'], dtype=object)